In [1]:
import pickle
import os
import numpy as np
print("PYTHONPATH:", os.environ.get('PYTHONPATH'))
print("PATH:", os.environ.get('PATH'))
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import pandas as pd
import phate
import math
import random
import gc
import scprep
from datetime import datetime, time
from matplotlib.animation import ImageMagickWriter
import matplotlib.animation as animation
import zipfile
from urllib.request import urlopen
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import gaussian_kde
from scipy.stats import kde
from scipy.stats import binned_statistic
from scipy.stats import f_oneway
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['pdf.fonttype'] = 42
print(sns.__version__)
from anndata import AnnData
import scanpy as sc
from delve import *
import anndata as ad
from sklearn.preprocessing import MinMaxScaler
from kh import sketch
from sklearn.cluster import KMeans
import umap
print(sc.__version__)
today = datetime.now().strftime("%m%d%Y-%H%M")

PYTHONPATH: None
PATH: c:\Users\gases\.conda\envs\python_3_7;C:\Users\gases\.conda\envs\python_3_7;C:\Users\gases\.conda\envs\python_3_7\Library\mingw-w64\bin;C:\Users\gases\.conda\envs\python_3_7\Library\usr\bin;C:\Users\gases\.conda\envs\python_3_7\Library\bin;C:\Users\gases\.conda\envs\python_3_7\Scripts;C:\Users\gases\.conda\envs\python_3_7\bin;C:\ProgramData\Anaconda3\condabin;C:\ProgramData\Anaconda3\lib\site-packages\PyQt5\Qt5\bin;C:\ProgramData\Anaconda3;C:\ProgramData\Anaconda3\Library\mingw-w64\bin;C:\ProgramData\Anaconda3\Library\usr\bin;C:\ProgramData\Anaconda3\Library\bin;C:\ProgramData\Anaconda3\Scripts;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\libnvvp;.;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program F

In [ ]:
### full_dir should be the directory that contains the output of your "calculate cell properties" notebook. Called cell_data by default
### well_list should be every well in the dataset that you intend to combine into a single adata object for analysis

full_dir = r'path\to\my\data'
well_list = ['B02','B03','B04','B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11']

In [ ]:
# Definition to Normalize the dataframe by z-score

def standardizeColumns(df):
    df = df.copy()
    df.iloc[:,:] = df.iloc[:,:].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
    return df

In [ ]:
#Builds a dataframe for each well defined in well_list, then outputs the length of that list. The number output is how many cells
#were found in the well

fullest_df = pd.DataFrame()
for well in well_list:
    print(f'starting Well {well}')
    full_df = pd.read_csv(os.path.join(full_dir,f'cell_data_{well}_df.csv'), sep=',') 

   ### This part here is optional, you can add important metadata information at this step. Treatment and Sample ID are shown as examples
    if '2' in well and ('B' in well or 'C' in well):
        full_df['treatment'] = 'control' 
        full_df['sample_ID'] = 1
    if '3' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '2day' 
       full_df['sample_ID'] = 2
    if '4' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '5day' 
       full_df['sample_ID'] = 3
    if '5' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '9day' 
       full_df['sample_ID'] = 4
    if '6' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '11day' 
       full_df['sample_ID'] = 5
    if '7' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '13day' 
       full_df['sample_ID'] = 6
    if '8' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '15day' 
       full_df['sample_ID'] = 7
    if '9' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '17day' 
       full_df['sample_ID'] = 8
    if '10' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '20day' 
       full_df['sample_ID'] = 9
    if '11' in well and ('B' in well or 'C' in well):
       full_df['treatment'] = '22day' 
       full_df['sample_ID'] = 10
    if '2' in well and ('D' in well or 'E' in well):
       full_df['treatment'] = '24day' 
       full_df['sample_ID'] = 11
    if '3' in well and ('D' in well or 'E' in well):
       full_df['treatment'] = '27day' 
       full_df['sample_ID'] = 12
    if '4' in well and ('D' in well or 'E' in well):
       full_df['treatment'] = '29day' 
       full_df['sample_ID'] = 13
    if '5' in well and ('D' in well or 'E' in well):
       full_df['treatment'] = '31day' 
       full_df['sample_ID'] = 14
       
    exec(f'well{well}_df = full_df.copy()')
    fullest_df = pd.concat([fullest_df, full_df], ignore_index = True)
    print(len(full_df))
    print(len(fullest_df))

### You may find that you need to force some of the columns added this way into specific data types. You can do that here.
# Convert sample_ID column to categorical data type
fullest_df['sample_ID'] = fullest_df['sample_ID'].astype('category')

In [ ]:
### It is good practice to save these dataframes at key steps, such as this one
fullest_df.to_csv(r'path\to\my\data)

In [ ]:
# Drop columns that you don't need - Preparation for conversion to AnnData object
fullest_df = fullest_df.drop(columns=["Unnamed: 0", "bbox-0", "bbox-1", "bbox-2", etc...])
# Extract metadata columns and store them in a separate dataframe
metadata = fullest_df[["label", "well", "sample_ID", "treatment"]]
# Remove metadata columns from the main dataframe
fullest_df = fullest_df.drop(columns=["label", "well", "sample_ID", "treatment"])

In [ ]:
#Z normalize the data
standard_df = standardizeColumns(fullest_df)

In [ ]:
#min-max normalize the data - Not used in this example, provided for potential use
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(fullest_df)
normalized_data = pd.DataFrame(normalized_data, index = fullest_df.index, columns = fullest_df.columns)

In [ ]:
# Convert the pandas dataframe to an anndata object
standard_adata = ad.AnnData(standard_df)
# Add metadata back to the anndata object
standard_adata.obs = metadata.copy()
standard_adata.obs_names = [f'c_{i}' for i in standard_adata.obs_names]

#Save the entire adata file
adata_save_path = r'my\save\path\standard_adata.h5ad'
standard_adata.write_h5ad(adata_save_path)

In [ ]:
###Sketching lets your subsample your data accurately. 
### This example groups for subsampling based on the sample_id metadata

idx, standard_adata_sub = sketch(standard_adata, num_subsamples = 1200, frequency_seed = 42, sample_set_key = 'sample_ID')
#Save the entire adata file
adata_save_path = r'my\save\path\standard_adata_sub_sub.h5ad'
standard_adata_sub.write_h5ad(adata_save_path)

Subsequent notebooks will load in the saved adata file of your choice, either the full dataset of the subsampled dataset.
Additionally, you can produce any desired normalized or non-normalized adata file here. Z-score normalized is the provided example, but other methods can be used with minor modifications. i.e. min-max normalization using the provided code.